In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
import concurrent
import requests
import os
import os.path

In [ ]:
def do_scrap(link, category):
    key = '1783319260d70f84da21868ce0fd6207'

    payload = {'api_key': key, 'url': link, 'keep_headers': 'true'}
    head = {
        'authority': 'api.tracker.gg',
        'method': 'GET',
        'path': '/api/v1/valorant/lfg/search?region=any&playlist=competitive&skill=any',
        'scheme': 'https',
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
        'accept-encoding': 'gzip, deflate, br',
        'accept-language': 'nb-NO,nb;q=0.9,no;q=0.8,en-US;q=0.7,en;q=0.6,ru;q=0.5',
        'cache-control': 'max-age=0',
        'cookie': '__cflb=02DiuFQAkRrzD1P1mdkJhfdTc9AmTWwYkL5dhc6BaWz7E; X-Mapping-Server=s14; __cf_bm=eCW52EohIqvzfugOyKvNvo4spvffbOwnBy5YZBvh4gQ-1653777883-0-AYz5PvgOpqpL7/UCjWN7/dokINYUw/EV1D4T6eY5n9YecMF2lY0M4TWWdDyySEalQ3lzxua3IYwD9KUjzZtmFdM=',
        'sec-ch-ua': 'Not A;Brand";v="99", "Chromium";v="101", "Google Chrome";v="101',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': 'macOS',
        'sec-fetch-dest': 'document',
        'sec-fetch-mode': 'navigate',
        'sec-fetch-site': 'none',
        'sec-fetch-user': '?1',
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.0.0 Safari/537.36'
        }
    count = 0
    path = f'excel_dir/{category}.csv'
    if not os.path.exists(path):
        while True:
            response = requests.get('http://api.scraperapi.com', params=payload, headers=head)
            if response.status_code == 200:
                print(f'Success:',response.status_code)
                return response
            else:
                print(f'Failure:',response.status_code)
                count += ''
                if count >= 2:
                    return None
                continue
    else:
        print(f'\t{category} has already been scrapped')
        return None
#200	Successful response. - ScraperAPI
#400    Bad request is sent when the server cannot understand the request sent by the client. - Requests
#401    Unauthorized is sent whenever fulfilling the requests requires supplying valid credentials. - Requests
#403    Forbidden means that the server understood the request but will not fulfill it. - Requests
#404	Page requested does not exist. - ScraperAPI
#410	Page requested is no longer available. - ScraperAPI
#500	After retrying for 60 seconds, the API was unable to receive a successful response. - ScraperAPI
#429	You are sending requests too fast, and exceeding your concurrency limit. - ScraperAPI
#403	You have used up all your API credits. - ScraperAPI

In [ ]:
def write(response, category):
# Write html file from data scrapped
    with open(f"html_dir/{category}.html","w",encoding="utf-8")as html_file:
        html_file.write(response.text)

In [ ]:
def read(category):    
    # Read html file from scrapped data
    with open(f"html_dir/{category}.html","r",encoding='utf-8')as html_file:
        output=html_file.read()
        
        soup = BeautifulSoup(output, 'lxml')
        if category == 'skjermer' or category == 'skjermkort':
            products = soup.find_all('div', class_ = 'product-list-item subscription-price-visible')
        else:
            products = soup.find_all('div', class_ = 'product-list-item')
            
        product_dict = {}
        for product in products:
            link = str('komplett.no' + product.a['href'])
            image = str('komplett.no/img/p/800/' + product.a['href'].split('/')[2] + '.jpg')
            sale = None
            price_now = product.find('span', class_= 'product-price-now')
            if price_now != None:
                price_now = price_now.text
                price_now = int(''.join(char for char in price_now if char.isalnum()))
                price_before = product.find('div', class_= 'product-price-before')
                if price_before != None:
                    price_before = product.find('div', class_= 'product-price-before').text.replace('Før', '').replace(',-', '').strip()
                    price_before = int(''.join(char for char in price_before if char.isalnum()))
                    sale = True
            
            name = product.h2.text.replace('å', 'aa').replace('ø', 'o')
            stats = product.p.text.replace(',','').split()
            #available = product.find('span', class_='stockstatus-stock-details').text.replace('å', 'aa').replace('ø', 'o').split(' ')[0]
            try:
                available = product.find('span', class_='stockstatus-stock-details').text.replace('å', 'aa').replace('ø', 'o').split(' ')[0].replace('+', '')
                if available == 'Ikke' or available == 'Bestillingsvare.':
                    available = 0
            except AttributeError as error:
                    available = error
                    
            itemnumber = product.find('div', class_='product-data').text.replace(' ', '').replace('\n', '').split('/')[0].split(':')[1]

            product_dict[itemnumber] = [name, price_now, price_before, sale, available, stats, image, link, 'komplett', category]       
        
        return product_dict

In [ ]:
def komplett_scrap(links):

    # Check if we have the correct directories
    if not os.path.exists('html_dir'):
        os.mkdir('html_dir')
    if not os.path.exists('excel_dir'):
        os.mkdir('excel_dir')

    for link in links:
        
        # Category on the item in our current link
        category = link.split('?')[0].split('/')[-1].replace('-','_')
        print(category)
        # Write down the data we get from scrapping
        scrapped = do_scrap(link, category)
        if scrapped != None:
            print('\tScrap: Success!')
            print('\tWriting...')
            write(scrapped, category)
            print('\tReading...')
            current = read(category)
            # Used for columns in excel
            col = ['name', 'price_now', 'price_before', 'sale', 'available', 'stats', 'image', 'link', 'site', 'category']
            # Create dataframe
            df = pd.DataFrame(current.values(), columns = col)
            #df = df.loc[df['sale']==True]
            # Convert dataframe to excel file
            df.to_csv(f'excel_dir/{category}.csv')
            print(f'\tScrapped: {category}')

In [ ]:
links = [
        'https://www.komplett.no/category/12831/hvitevarer/stekeovner?nlevel=10639%C2%A712831&hits=48',
        'https://www.komplett.no/category/12839/hvitevarer/kjoeleskap?',
        'https://www.komplett.no/category/21169/mobiler-klokker/smartklokker?nlevel=10444%C2%A721169&hits=120',
        'https://www.komplett.no/category/21064/mobiler-klokker/mobiltelefoner?nlevel=10444%C2%A721064&hits=216',
        'https://www.komplett.no/category/10088/datautstyr/lagring/harddiskerssd?nlevel=10000%C2%A728001%C2%A710088&hits=288',
        'https://www.komplett.no/category/12840/hvitevarer/frysere',
        'https://www.komplett.no/category/142/foto-video/kameraer?nlevel=10190%C2%A7142&hits=48',
        'https://www.komplett.no/category/736/hjem-fritid/kaffemaskiner?nlevel=10560%C2%A7736&hits=120',
        'https://www.komplett.no/category/12817/hvitevarer/vaskemaskiner?nlevel=10639%C2%A712817&hits=48',
        'https://www.komplett.no/category/12821/hvitevarer/toerketromler',
        'https://www.komplett.no/category/10462/datautstyr/pc-komponenter/vifter/kjoeling/vannkjoeling?nlevel=10000%C2%A728003%C2%A710462&hits=288',
        'https://www.komplett.no/category/753/hvitevarer/mikroboelgeovner?',
        'https://www.komplett.no/category/10057/datautstyr/pc-komponenter/stroemforsyning?nlevel=10000%C2%A728003%C2%A710057&hits=72',
        'https://www.komplett.no/category/12824/hvitevarer/oppvaskmaskiner?nlevel=10639%C2%A712824&hits=48',
        'https://www.komplett.no/category/11157/tv-lyd-bilde/tv-video/tv-er?nlevel=10719%C2%A730000%C2%A711157&hits=240',
        'https://www.komplett.no/category/11158/datautstyr/skjermer/skjermer?nlevel=10000%C2%A710392%C2%A711158&hits=264',
        'https://www.komplett.no/category/21635/gaming/gaming-utstyr/gaming-tastatur?nlevel=10431%C2%A721603%C2%A721635&hits=168',
        'https://www.komplett.no/category/199889/hjem-fritid/stoevsugere-rengjoering?nlevel=10560%C2%A7199889&hits=120',
        'https://www.komplett.no/category/21605/gaming/gaming-utstyr/gaming-mus?nlevel=10431%C2%A721603%C2%A721605&hits=168',
        'https://www.komplett.no/category/21640/gaming/gaming-utstyr/gaming-headset?nlevel=10431%C2%A721603%C2%A721640&hits=192',
        'https://www.komplett.no/category/21607/gaming/gaming-utstyr/gaming-musematte?nlevel=10431%C2%A721603%C2%A721607&hits=72',
        'https://www.komplett.no/category/21650/gaming/gaming-utstyr/spillkontrollere?hits=120',
        'https://www.komplett.no/category/11204/datautstyr/pc-komponenter/prosessorer?nlevel=10000%C2%A728003%C2%A711204&hits=144',
        'https://www.komplett.no/category/10111/datautstyr/pc-komponenter/hovedkort?nlevel=10000%C2%A728003%C2%A710111&hits=240',
        'https://www.komplett.no/category/10412/datautstyr/pc-komponenter/skjermkort?nlevel=10000%C2%A728003%C2%A710412&hits=360',
        ]

In [101]:
komplett_scrap(links)

stekeovner
	stekeovner has already been scrapped
kjoeleskap
	kjoeleskap has already been scrapped
smartklokker
	smartklokker has already been scrapped
mobiltelefoner
	mobiltelefoner has already been scrapped
harddiskerssd
	harddiskerssd has already been scrapped
frysere
	frysere has already been scrapped
kameraer
	kameraer has already been scrapped
kaffemaskiner
	kaffemaskiner has already been scrapped
vaskemaskiner
	vaskemaskiner has already been scrapped
toerketromler
	toerketromler has already been scrapped
vannkjoeling
	vannkjoeling has already been scrapped
mikroboelgeovner
	mikroboelgeovner has already been scrapped
stroemforsyning
	stroemforsyning has already been scrapped
oppvaskmaskiner
	oppvaskmaskiner has already been scrapped
tv_er
	tv_er has already been scrapped
skjermer
	skjermer has already been scrapped
gaming_tastatur
	gaming_tastatur has already been scrapped
stoevsugere_rengjoering
	stoevsugere_rengjoering has already been scrapped
gaming_mus
	gaming_mus has already